In [ ]:
# https://github.com/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb

# <b> Custom Training StyleGan2-ADA </b>
StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

You're going to need at least 12 GB of RAM for the stylegan-2 to run. With Google Collab, you'd need pro or pro plus unfortunatley.

In [ ]:
!pip install numpy==1.19.5 # For some reason, need to downgrade to run.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.7
Requires: google-pasta, termcolor, keras-applications, six, tensorflow-estimator, opt-einsum, protobuf, absl-py, grpcio, wheel, wrapt, astor, tensorboard, gast, keras-preprocessing, numpy
Required-by: stable-baselines, magenta, kapre


In [ ]:
!nvidia-smi

Sun Mar 13 04:51:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# <b> Install Repo to Google Drive </b>
Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [ ]:
import os
if os.path.isdir("/content/drive/My Drive/colab-sg2-ada"):
    %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/My Drive/colab-sg2-ada/stylegan2-ada


# <b> Convert dataset to .tfrecords </b>
<i> Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells. </i>

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [ ]:
# #if you manually uploaded your dataset to Colab, unzip it
# zip_path = "/content/CAT1024.zip"

# !unzip {zip_path} -d /content/

Now that your image dataset is uploaded, we need to convert it to the .tfrecords format.

Depending on the resolution of your images and how many you have, this can take a while.

In [ ]:
#update this to the path to your image folder
dataset_path = "/content/drive/MyDrive/abstract" #/content/drive/MyDrive/abstract abstract_tf for entire ~7500 images.

#give your dataset a name
dataset_name = "abstract_tf" #abstract_tf for entire ~7500 images.

#you don't need to edit anything here
!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

"""NOTE: I move the ./datasets/dataset_name/ to drive/My Drive so I don't have to rerun this cell."

Loading images from "/content/drive/MyDrive/abstract_1000"
Creating dataset "./datasets/abstract_tf_1000"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 100 images.


<b> Train a custom model </b>
We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [ ]:
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:
#this name must EXACTLY match the dataset name you used when creating the .tfrecords file
dataset_name = "abstract_tf_1000" # abstract_tf
#how often should the model generate samples and a .pkl file
snapshot_count = 5
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = True
"""Since mirrored vertically and horizontally, we now have x4 as many records."""

#metrics? 
metric_list = None
#augments
augs = "bgcfnc"

#
# this is the most important cell to update
#
# running it for the first time? set it to ffhq(+resolution)
# resuming? get the path to your latest .pkl file and use that
# resume_from = "/content/drive/My\ Drive/colab-sg2-ada2/stylegan2-ada/results/00008-dante1024-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000160.pkl"
# resume_from = 'ffhq256' --resume --resume={resume_from} 
#!python train.py --outdir /content/drive/MyDrive/results_abstract_gan --snap={snapshot_count} --cfg=auto --data=/content/drive/MyDrive/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list}
!python train.py --outdir=/content/drive/MyDrive/results_abstract_gan --snap={snapshot_count} --gpus=1 --data=/content/drive/MyDrive/{dataset_name} --kimg=3000 --aug=ada

tcmalloc: large alloc 4294967296 bytes == 0x55814b8d8000 @  0x7f4804900001 0x7f4801b0354f 0x7f4801b53b58 0x7f4801b57b17 0x7f4801bf6203 0x558143b6f424 0x558143b6f120 0x558143be3b80 0x558143bde66e 0x558143b7136c 0x558143bb27b9 0x558143baf6d4 0x558143b6fc29 0x558143be3e61 0x558143bde02f 0x558143aafe2b 0x558143be0633 0x558143bde02f 0x558143aafe2b 0x558143be0633 0x558143bde66e 0x558143aafe2b 0x558143be0633 0x558143b709da 0x558143bdeeae 0x558143bde02f 0x558143bddd43 0x558143ca8302 0x558143ca867d 0x558143ca8526 0x558143c801d3
tcmalloc: large alloc 4294967296 bytes == 0x55824b8d8000 @  0x7f48048fe1e7 0x7f4801b0346e 0x7f4801b53c7b 0x7f4801b5435f 0x7f4801bf6103 0x558143b6f424 0x558143b6f120 0x558143be3b80 0x558143bde02f 0x558143b70aba 0x558143bdfcd4 0x558143bde02f 0x558143b70aba 0x558143bdfcd4 0x558143bde02f 0x558143b70aba 0x558143bdfcd4 0x558143b709da 0x558143bdeeae 0x558143bde02f 0x558143b70aba 0x558143be32c0 0x558143bde02f 0x558143b70aba 0x558143bdfcd4 0x558143bde66e 0x558143b7136c 0x558143bb

While it’s training...
Once the above cell is running you should be training!

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you snapshot_count setting you should see the results folder in your Google drive folder fill with both samples (fakesXXXXXx.jpg) and model weights (network-snapshot-XXXXXX.pkl). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the resume_from path to continue training from the latest model.